In [1]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [4]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/opencv/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['01.jpg',
 '02.jpg',
 '03.jpg',
 '04.jpg',
 'dataimage_180.jpg',
 'models',
 '27.ipynb',
 '30.ipynb',
 '28.ipynb',
 '29.ipynb',
 'FaceImage',
 'done_face_features.pickle',
 '31c_data_preprocessing.ipynb']

In [5]:
# 1. DATA

In [8]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/opencv/done_face_features.pickle", mode="rb"))

In [9]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.04440678  0.21475683  0.01395946 ... -0.0886332   0.06150215
   -0.00736863]]

 [[ 0.04842703  0.20101416  0.0238631  ... -0.08374722  0.07649732
    0.00379119]]

 [[ 0.05084311  0.17329712 -0.01429705 ... -0.06301611  0.11193115
    0.02131533]]

 ...

 [[ 0.04327805  0.23951319 -0.08468571 ... -0.00060132  0.08849992
    0.00982566]]

 [[ 0.06792032  0.20525679 -0.09386639 ... -0.08474214  0.04895966
    0.01614708]]

 [[ 0.03505763  0.17826444 -0.08103755 ...  0.00479291  0.07760584
    0.08672322]]]
['1721031561_NguyenDucHieu' '1721031561_NguyenDucHieu'
 '1721031561_NguyenDucHieu' ... '162000339_HuynhAnhDuy'
 '162000339_HuynhAnhDuy' '162000339_HuynhAnhDuy']


In [10]:
x.shape

(4694, 1, 128)

In [11]:
y.shape

(4694,)

In [12]:
x = x.reshape(-1, 128)

In [13]:
x.shape

(4694, 128)

In [14]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, random_state=1)

In [15]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4224, 128), (470, 128), (4224,), (470,))

In [16]:
# 2. Train machine learning

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [18]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [19]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [20]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.73
Accurency Test = 0.70
F1 Score Train = 0.53
F1 Score Test = 0.53


In [21]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [22]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.92
Accurency Test = 0.82
F1 Score Train = 0.84
F1 Score Test = 0.74


In [23]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [24]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.76
F1 Score Train = 1.00
F1 Score Test = 0.67


In [25]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [26]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [27]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.82
F1 Score Train = 0.97
F1 Score Test = 0.74


In [28]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [29]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  11.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  14.7s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   9.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=  11.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   7.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   8.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.8s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [30]:
model_best_estimator = model_grid.best_estimator_

In [31]:
model_grid.best_score_

0.8532196969696969

In [33]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/opencv/ml_face_person_identity.pkl", mode='wb'))